<a href="https://colab.research.google.com/github/Hritik1100/Depression-Prediction-using-ML/blob/main/Depression_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Depression Prediction on Twitter using Machine Learning Algorithms**

Import data and download required libraries

In [ ]:
!pip install git+https://github.com/casics/spiral.git  #Spiral
!pip install contractions   #Word Contraction
!pip install -U git+https://github.com/ray-project/tune-sklearn.git && pip install 'ray[tune]'  #TuneSearch
!!pip install scikit-optimize
!pip install ConfigSpace

In [ ]:
import pandas as pd
import re
import nltk
import sklearn
data = pd.read_csv('dataset.csv',encoding='latin-1')

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

In [ ]:
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
import string
lem = WordNetLemmatizer()
corpus_test = []
stopwords = set(stopwords.words('english'))
punc = string.punctuation

Preprocessing

In [ ]:
corpus=[]
import contractions
from spiral import ronin
for i in range(len(data)):
  review = re.sub(r"http\S+", "", data['message'][i]) # remove urls
  review = re.sub(r'<([^>]*)>', ' ', review) # remove emojis
  review = re.sub(r'@\w+', ' ', review) # remove @ mentions
  review = re.sub(r'#', '', review) # remove hashtag symbol
  review = re.sub(r'[0-9]+', ' ', review) # remove numbers
  review = re.sub(r'[^A-Za-z0-9,?.!]+', ' ', review)
  review = review.lower()
  l=[]
  for word in review.split():
    l.append(contractions.fix(word))      #you're ------> you are
  review = ' '.join(l)
  review = ronin.split(review)            #mentalhealth----> mental health
  review = ' '.join(review)
  review = nltk.word_tokenize(review)
  review = [lem.lemmatize(word) for word in review if word not in stopwords and word not in punc]
  review = ' '.join(review)
  corpus.append(review)

Splitting data into Train and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X = corpus
y = data['label']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0) #70:30

Feature Engineering (Using Count Vectorizer)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer    #Count Vectorizer
fun = CountVectorizer(max_features=1500)
X_train_vector = fun.fit_transform(X_train).toarray()
X_test_vector = fun.transform(X_test).toarray()

Hyper Parameters

In [ ]:
params = {'clf__learning_rate':[0.2,0.3],  #Gradient Boost Hyperparameters
          'clf__n_estimators':[150,250]}


Finding the right Hyperparameters using TuneSearchCV

In [ ]:
from sklearn.metrics import precision_score,confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
from tune_sklearn import TuneSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'weighted'),
           'recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted')
}

pipeline = Pipeline([('Smote',SMOTE(random_state=42)),('clf',GradientBoostingClassifier())])

##5 fold Cross Validation
tune_imba = TuneSearchCV(pipeline,params,scoring=scoring,max_iters=10,n_jobs=-1,search_optimization = 'bayesian',cv=5,verbose=1,refit='f1')

tune_imba.fit(X_train_vector,y_train)
y_pred = tune_imba.predict(X_test_vector)
print('Validation F1-Score',tune_imba.best_score_)
print("Test F1-score - ", f1_score(y_test, y_pred))
print("Test precision score - ", precision_score(y_test, y_pred))
print("Test recall score - ", recall_score(y_test, y_pred))
print("Test accuracy score - ", accuracy_score(y_test, y_pred))
tune_imba.best_params_

INFO:ray.tune.tune:Total run time: 6639.06 seconds (6636.05 seconds for the tuning loop).
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Validation F1-Score 0.9328491177116709
Test F1-score -  0.8480189237137788
Test precision score -  0.9215938303341902
Test recall score -  0.7853231106243155
Test accuracy score -  0.9366214549938348


{'clf__learning_rate': 0.23683576691451325, 'clf__n_estimators': 186}